In [1]:
import traceback
import asyncio
import aiohttp

from jsonpath import jsonpath
import pandas as pd
import sqlalchemy
from sqlalchemy import text
from sqlalchemy.dialects.mysql import insert
import pymongo
import hmac
import hashlib

from test_func import data

In [3]:
trade_api1 = {
    'api_key': 'emLKz5PIwkhFfVJefFtIPQnZF37Hjy0swD9LtZlNRgb7LLjVeH1dg1SyQcvUxFEw',
    'secret_key': 'ASJymGaO90FfTplCLd8DTfCTYNykNFmif4lHMwsIhSI0dBc6tC6zlnrfUcaRDmlQ'
}
headers = {'X-MBX-APIKEY': trade_api1['api_key']}
session = aiohttp.ClientSession(headers=headers)

In [4]:
async with session.get(
        "https://fapi.binance.com/fapi/v1/ticker/24hr", 
        ) as res:
    print(await res.text())

[{"symbol":"DFUSDT","priceChange":"-0.0009800","priceChangePercent":"-1.419","weightedAvgPrice":"0.0691098","lastPrice":"0.0680800","lastQty":"85","openPrice":"0.0690600","highPrice":"0.0713000","lowPrice":"0.0670600","volume":"139992204","quoteVolume":"9674838.7321000","openTime":1736844660000,"closeTime":1736931109389,"firstId":5182114,"lastId":5298943,"count":116813},{"symbol":"POWRUSDT","priceChange":"0.0023000","priceChangePercent":"0.837","weightedAvgPrice":"0.2807342","lastPrice":"0.2770000","lastQty":"8094","openPrice":"0.2747000","highPrice":"0.2888000","lowPrice":"0.2737000","volume":"83881187","quoteVolume":"23548316.4161000","openTime":1736844660000,"closeTime":1736931108011,"firstId":91287085,"lastId":91427251,"count":140164},{"symbol":"EOSUSDT","priceChange":"0.050","priceChangePercent":"6.443","weightedAvgPrice":"0.798","lastPrice":"0.826","lastQty":"5.4","openPrice":"0.776","highPrice":"0.835","lowPrice":"0.772","volume":"91373896.3","quoteVolume":"72924711.287","openTi

In [4]:
async with session.get("https://fapi.binance.com/fapi/v1/ping") as res:
    print(await res.text())

{}


In [69]:
async with session.get(
        "https://fapi.binance.com/fapi/v1/exchangeInfo", 
        data=params
        ) as res:
    exchange_info_dict = await res.json()
    
data = jsonpath(exchange_info_dict, '$.symbols.[symbol,pricePrecision]')
data = [data[0::2], data[1::2]]
sr_price_precision = pd.DataFrame(data, index=['symbol', 'pricePrecision']).T.set_index('symbol')['pricePrecision']

In [7]:
async with session.get(
        "https://fapi.binance.com/fapi/v1/time", 
        ) as res:
    server_time = (await res.json())['serverTime']
params = [('timestamp', str(server_time))]
signature = hmac.new(
    trade_api1['secret_key'].encode(),
    "&".join(["=".join(param) for param in params]).encode(),
    hashlib.sha256
).hexdigest()
params.append(('signature', signature))

async with session.get(
        "https://fapi.binance.com/fapi/v2/positionRisk", 
        data=params
        ) as res:
    df_pos_org = pd.DataFrame(await res.json()).set_index('symbol')

In [28]:
async with session.get(
        "https://fapi.binance.com/fapi/v1/time", 
        ) as res:
    server_time = (await res.json())['serverTime']
params = [('timestamp', str(server_time))]
signature = hmac.new(
    trade_api1['secret_key'].encode(),
    "&".join(["=".join(param) for param in params]).encode(),
    hashlib.sha256
).hexdigest()
params.append(('signature', signature))

async with session.get(
        "https://fapi.binance.com/fapi/v2/positionRisk", 
        data=params
        ) as res:
    df_pos_org = pd.DataFrame(await res.json()).set_index('symbol')

df_pos = df_pos_org.astype({
    'positionAmt': float, 
    'entryPrice': float, 
    'breakEvenPrice': float, 
    'markPrice': float,
    'unRealizedProfit': float, 
    'liquidationPrice': float, 
    'leverage': int,
    'maxNotionalValue': float,
    'notional': float, 
    'isolatedWallet': int, 
    'updateTime': int,
    'adlQuantile': float
})
df_pos = df_pos[df_pos['positionAmt']!=0]

async with session.get(
        "https://fapi.binance.com/fapi/v1/exchangeInfo"
) as res:
    df_exchange_info = pd.DataFrame((await res.json())['symbols']).set_index('symbol')

async with session.get(
        "https://fapi.binance.com/fapi/v1/ticker/bookTicker", 
) as res:
    df_ticker = pd.DataFrame(await res.json()).set_index('symbol')

for symbol, row in df_pos.iterrows():
    curr_side = 'BUY' if row['positionAmt'] > 0 else 'SELL'
    digit = df_exchange_info.loc[symbol, 'pricePrecision']
    pos = df_pos_org.loc[symbol, 'positionAmt']
    if curr_side == 'BUY':
        stop_price = round(float(df_ticker.loc[symbol, 'askPrice']) * 1.005, digit)
        side = 'SELL'
    elif curr_side == 'SELL':
        stop_price = round(float(df_ticker.loc[symbol, 'bidPrice']) * 0.995, digit)
        side = 'BUY'
    
    order = {
        'symbol': symbol,
        'type': 'TAKE_PROFIT_MARKET',
        'quantity': str(pos.strip('-')),
        'side': side,
        'stopPrice': str(stop_price)
    }
    params = list(order.items())
    async with session.get(
        "https://fapi.binance.com/fapi/v1/time", 
        ) as res:
        server_time = (await res.json())['serverTime']
    params.append(('timestamp', str(server_time)))
    params.sort(key=lambda x: x[0])
    signature = hmac.new(
        trade_api1['secret_key'].encode(),
        "&".join(["=".join(param) for param in params]).encode(),
        hashlib.sha256
    ).hexdigest()
    params.append(('signature', signature))
    async with session.post(
            'https://fapi.binance.com/fapi/v1/order',
            data=params, timeout=60
    ) as res:
            text = await res.text()
            print(f"Order {order['symbol']}: {res.status}\n{text}")

ContentTypeError: 0, message='Attempt to decode JSON with unexpected mimetype: text/html', url=URL('https://fapi.binance.com/fapi/v2/positionRisk')

In [ ]:
session.put('https://fapi.binance.com/fapi/v1/listenKey', params={'listenKey': listen_key})

In [1]:
import asyncio
import aiohttp
import websockets

In [17]:
trade_api1 = {
        'api_key': 'emLKz5PIwkhFfVJefFtIPQnZF37Hjy0swD9LtZlNRgb7LLjVeH1dg1SyQcvUxFEw',
        'secret_key': 'ASJymGaO90FfTplCLd8DTfCTYNykNFmif4lHMwsIhSI0dBc6tC6zlnrfUcaRDmlQ'
}

In [3]:
headers = {'X-MBX-APIKEY': trade_api1['api_key']}
session = aiohttp.ClientSession(headers=headers)

In [6]:
res = await session.post('https://fapi.binance.com/fapi/v1/listenKey')
listen_key = (await res.json())['listenKey']

In [13]:
wss_uri = 'wss://fstream.binance.com/ws/{listenKey}'

In [17]:
wss_client = await websockets.connect(wss_uri.format(listenKey=listen_key))
try:
    message = await asyncio.wait_for(wss_client.recv(), timeout=10)
except asyncio.TimeoutError:
    print('Timeout')

Timeout


In [19]:
await session.put('https://fapi.binance.com/fapi/v1/listenKey', params={'listenKey': listen_key})

<ClientResponse(https://fapi.binance.com/fapi/v1/listenKey?listenKey=7PvYMje33qPd98CT7mDSJ1HO17XwTLiPWjuwfJx1lhyRT7En8YCV287UKGorcS4e) [200 OK]>
<CIMultiDictProxy('Date': 'Mon, 06 Jan 2025 09:33:59 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'Tengine', 'Vary': 'Accept-Encoding', 'x-mbx-used-weight-1m': '1', 'x-response-time': '2ms', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': '*,clienttype,Content-Type,lang,x-ui-request-trace,bnc-uuid', 'Content-Security-Policy': "default-src 'self'", 'Cache-Control': 'no-cache, no-store, must-revalidate', 'Expires': '0', 'Pragma': 'no-cache', 'strict-transport-security': 'max-age=31536000; includeSubdomains', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'Content-Encoding': 'gzip')>